In [ ]:
%html
<style>
table th, td { text-align: left; } 
</style>

# 第8章 フォーム（Form）

## 8.1 概要
### フォームの役割
- ユーザーの入力データを保持
- 入力データのバリデーション（妥当性チェック）を行い、妥当性検証済みのデータやエラーメッセージを保持

### ユーザーの入力データの保持
画面の入力フィールドの`name`属性とフォームクラスのクラス変数名を対応させて定義しておくことで、画面の入力内容からフォームオブジェクト、フォームオブジェクトから画面の入力内容へのデータ変換を容易に行うことが出来る

### 入力データのバリデーションと検証済みデータ・エラーメッセージの保持
フォームに適切な名前のバリデーションメソッドを用意しておくことで、バリデーション実行時にフォームが決まったルールに沿って、次々とバリデーションメソッドを呼び出す  
フォーム内部には、以下のようにデータが保存される
- バリデーションOK：<br />妥当性検証済みのデータ
- バリデーションNG：<br />エラーメッセージ

### フォーム利用のメリット
- ユーザーの入力データを（定義にしたがって）型変換してくれる
- バリデーションをビューから分離できる
- 入力データやエラーメッセージをテンプレートに簡単に表示できる
- ユーザー認証系のよくあるフォームはDjangoがデフォルトで用意してくれている

## 8.2 バリデーションの仕組み
### バリデーションのルール
エントリーポイント：<br />フォームの`is_valid`メソッド  
メソッドが実行されると、大まかに以下の順番でバリデーションを行う
1. フィールドに定義されたフィールド単体のバリデーション（文字種などの最低限の形式チェック）
1. フォームに定義されたフィールド単体のバリデーション（値の妥当性チェック）
1. フォームに定義された複数フィールド間のバリデーション

#### フィールドに定義されたバリデーション
- `to_python()`
- `validate()`
- `run_validators()`：<br />文字種チェックなどのバリデーションをフィールドの`validators`属性に定義している場合に実行される

#### フィールド単体に対するバリデーション
- `clean_<フィールド名>`のメソッド名で任意に用意する
- クラスで定義した順に実行する

#### 相関チェック・データベースとの整合性チェック
- `clean()`

#### 妥当性検証済みデータ
- `cleaned_data`：<br />`dict`型変数、妥当性検証済みデータを保持する<br />`is_valid()`メソッドを実行することでフォームオブジェクト内に変数を生成する

#### エラーメッセージ
- `_errors()`：<br />`list`型変数、バリデーションのエラーメッセージを保持する<br />`is_valid()`メソッドを実行することでフォームオブジェクト内に変数を生成する

## 8.3 フォームクラスの書き方
慣例的に、フォームはアプリケーションディレクトリの`forms.py`という名前のモジュールに記述する  
フォームクラスは`django.forms.Form`を継承し、内部に画面の入力フィールドに対応するクラス変数と任意のバリデーションを定義する

### 8.3.1 入力フィールドの定義
入力フィールドに対する変数には、`django.forms.fields.Field`クラスのサヴクラスを利用する
利用できる組み込みの`Field`クラスの全容は[公式ドキュメント](https://docs.djangoproject.com/ja/2.1/ref/forms/fields/#built-in-field-classes)参照

#### 利用可能な`Field`クラス（一部）

| Fieldクラス | 適用されるウィジェット | 画面に表示される入力フォールドの型 |
| :---------- | :--------------------- | :--------------------------------- |
| CharField | TextInput | テキストフィールド ｜
| EmailField | EmailInput | メールアドレスフィールド(`type="email"`) |
| BooleanField | CheckboxInput | チェックボックス |
| ChoiceField | Select | セレクトボックス |
| IntegerField | NumberInput | 数値フィールド（`type="number"`） |
| DateField | DateInput | テキストフィールド |
| TimeField | TimeInput | テキストフィールド |
| DateTimeField | DateTimeInput | テキストフィールド |
| FileField | ClearableFileInput | ファイル選択フィールド |


#### フォームクラスの例（`accounts/forms.py`）
~~~python
from django import form
from django.contrib.auth.forms import UsernameField

class LoginForm(forms.Form):
    """ログイン画面用のフォーム"""
    username = UsernameField(
        label = 'ユーザー名',
        max_length = 255,
    )
    
    password = forms.CharField(
        label = 'パスワード',
        strip = False,
        widget = forms.PasswordInput(render_value = True),
    )
~~~

#### ウィジェット（widget）
画面に表示される際の入力フィールドの型やプレースホルダ・CSSクラスなどを定義するオブジェエクト  
利用する`Field`クラスによってデフォルトで適用されるウィジェットはそれぞれ決まっている（[利用可能な`Field`クラス](#利用可能なFieldクラス（一部）)）  
デフォルトから変更したい場合は、フィールドの`widget`属性を変更する


### 8.3.2 バリデーションメソッドの定義
#### `clean_<フィールド名>`
入力値を`cleaned_data`から取得する  
バリデーションOKの場合は妥当性チェック済みの値をreturnし、`cleaned_data`に再セットする  
バリデーションNGの場合は`django.forms.ValidationError`をraiseし、フォームオブジェクト内部の値にエラーメッセージを追加する  

##### フィールド`username`のバリデーションメソッドの実装
~~~python
def clear_username(self):
    # 入力値はcleared_dataから取得
    username = self.cleared_data['username']
    # 入力チが3桁より短ければバリデーションエラー
    if len(username) < 3:
        raise forms.ValidationError('%(min_length)s文字以上で入力してください', params = {'min_length': 3})
    return username
~~~

#### `clean`メソッド
バリデーションOKの場合は、検証済みデータをreturnする必要はない  
バリデーションNGの場合は、`django.forms.ValidationError`をraiseすることで、エラーメッセージを内部の変数に追加する

##### `clean`メソッドの実装
~~~python
def clear(self):
    username = self.cleared_data['username']
    password = self.cleared_data['password']
    try:
        user = User.objects.get(username=username)
    expect ObjectDoesNotExist:
        raise forms.ValidationError("正しいユーザー名を入力してください")
    # パスワードはハッシュ化されているので、平分での検索は出来ない
    if not user.check_password(password):
       raise forms.ValidationError("正しいパスワードを入力してください")
~~~